In [1]:
import matplotlib
import pathlib
import numpy as np

import mne
print(mne.__version__)

0.24.1


In [2]:
matplotlib.use('Qt5Agg')    #To open graphs using Qt

## Reading raw data

The files variable stores the subject and thier run number in a list. For each subject we have 2 runs. So 10_1 means 1st run of subject AB10, while 12_2 means 2nd run for subject AB12.

We are currently loading a small(12/80) amount of files to avoid the excessive time taken while testing efficiency of different algorithms

In [3]:
#The files we will be loading
#files = ['10_1', '10_2', '12_1', '12_2', '13_1', '13_2', '14_1', '14_2', '15_1', '15_2', '16_1', '16_2', '21_1', '21_2', '22_1', '22_2', '23_1', '23_2', '28_1', '28_2', '31_1', '31_2', '32_1', '32_2', '33_1', '33_2', '34_1', '34_2', '35_1', '35_2', '46_1', '46_2', '47_1', '47_2', '48_1', '48_2', '49_1', '49_2', '50_1', '50_2', '69_1', '69_2', '70_1', '70_2', '71_1', '71_2', '72_1', '72_2']
files = ['10_1', '10_2', '12_1', '12_2', '13_1', '13_2', '28_1', '28_2', '31_1', '31_2', '32_1', '32_2']

print(len(files))

12


The path variable is a dictionary which stores the relative path of various files as key-value pairs.

The keys are elements of files variable from above and the corresponding values are the relative path of those files on device.

In [4]:
#Paths Sahil
path = {}
for f in files:
    path[f] = '../../EEG original dataset/sub-AB{}/eeg/sub-AB{}_task-gonogo_run-{}_eeg.set'.format(f[0:-2], f[0:-2], f[-1])
    
path_channels_tsv = '../../data\sub-AB10_eeg_sub-AB10_task-gonogo_run-1_channels.tsv'

In [5]:
# #Paths Arush
# path = {}
# for f in files:
#     path[f] = 'data/other subjects set files/sub-AB{}_eeg_sub-AB{}_task-gonogo_run-{}_eeg.set'.format(f[0:-2], f[0:-2], f[-1])
    
# path_channels_tsv = 'data/other subjects set files/sub-AB10_eeg_sub-AB10_task-gonogo_run-1_channels.tsv'

The unclean_raw variable is a dictionary that stores the raw data that we have imported.

In [ ]:
#Loading raw data
unclean_raw = {}
for f in files:
    unclean_raw[f] = mne.io.read_raw_eeglab(path[f], eog=(), preload=True, uint16_codec=None, verbose=None)

## Assigning channel types and filtering

##### Getting a list of channel types
The raw data did not contain any channel types. We however had a tsv file which contained all the channel types. So we imported that tsv and extracted the channel types from there.
The channel names were slightly differently named than the names accepted by mne so we made a dictionary(reassign_dic) to correct it.
For example, the tsv had a channel named HEO which is an eog channel.

In [8]:
#get type of channels from tsv file
import pandas as pd
channel_types_tsv = pd.read_csv(path_channels_tsv, delimiter = '\t')

def get_channel_types_dic(channel_types_tsv):
    channel_types = {}
    for i in range(channel_types_tsv.shape[0]):
        channel_types[channel_types_tsv['name'][i]] = channel_types_tsv['type'][i].lower()
    reassign_dic = {'HEO':'eog', 'VEO':'eog', 'R-Dia-X-(mm)':'misc', 'R-Dia-Y-(mm)':'misc'}
    for k in list(reassign_dic.keys()):
        if k in list(channel_types):
            channel_types[k] = reassign_dic[k]
    return channel_types

channel_types = get_channel_types_dic(channel_types_tsv)
channel_types

{'AF3': 'eeg',
 'AF4': 'eeg',
 'F7': 'eeg',
 'F5': 'eeg',
 'F3': 'eeg',
 'F1': 'eeg',
 'Fz': 'eeg',
 'F2': 'eeg',
 'F4': 'eeg',
 'F6': 'eeg',
 'F8': 'eeg',
 'FT7': 'eeg',
 'FC5': 'eeg',
 'FC3': 'eeg',
 'FC1': 'eeg',
 'FCz': 'eeg',
 'FC2': 'eeg',
 'FC4': 'eeg',
 'FC6': 'eeg',
 'FT8': 'eeg',
 'T7': 'eeg',
 'C5': 'eeg',
 'C3': 'eeg',
 'C1': 'eeg',
 'Cz': 'eeg',
 'C2': 'eeg',
 'C4': 'eeg',
 'C6': 'eeg',
 'T8': 'eeg',
 'M1': 'eeg',
 'TP7': 'eeg',
 'CP5': 'eeg',
 'CP3': 'eeg',
 'CP1': 'eeg',
 'CPz': 'eeg',
 'CP2': 'eeg',
 'CP4': 'eeg',
 'CP6': 'eeg',
 'TP8': 'eeg',
 'M2': 'eeg',
 'P7': 'eeg',
 'P5': 'eeg',
 'P3': 'eeg',
 'P1': 'eeg',
 'Pz': 'eeg',
 'P2': 'eeg',
 'P4': 'eeg',
 'P6': 'eeg',
 'P8': 'eeg',
 'PO7': 'eeg',
 'PO5': 'eeg',
 'PO3': 'eeg',
 'POz': 'eeg',
 'PO4': 'eeg',
 'PO6': 'eeg',
 'PO8': 'eeg',
 'CB1': 'eeg',
 'O1': 'eeg',
 'Oz': 'eeg',
 'O2': 'eeg',
 'CB2': 'eeg',
 'VEO': 'eog',
 'HEO': 'eog',
 'EKG': 'ecg',
 'R-Dia-X-(mm)': 'misc',
 'R-Dia-Y-(mm)': 'misc'}

Now we use the .set_channel_types() method to assign the correct channel types to the raw data using the above dictionary.

In [60]:
#assigning channel types
for f in files:
    unclean_raw[f].set_channel_types(channel_types)

Adding a bandpass filter

In [10]:
#filtering the signals
for f in files:
    unclean_raw[f].filter(0.5, 60)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 60 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 60.00 Hz
- Upper transition bandwidth: 15.00 Hz (-6 dB cutoff frequency: 67.50 Hz)
- Filter length: 3301 samples (6.602 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 60 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband 

## Implementing ICA

In [14]:
#Creating copy
raw = {}
for f in files:
    raw[f] = unclean_raw[f].copy()

Creating ICA objects of n components for each subject and fitting

In [15]:
#Creating ICA objects
#pip install python-picard
import picard     #To use as a method for creating ICA objects
ica = {}
for f in files:
    ica[f] = mne.preprocessing.ICA(n_components = 21, method = 'picard', max_iter = 500, random_state = 42)

In [16]:
#Fitting ICA objects
for f in files:
    ica[f].fit(raw[f])

Fitting ICA to data using 61 channels (please be patient, this may take a while)
Selecting by number: 21 components
Fitting ICA took 41.8s.
Fitting ICA to data using 61 channels (please be patient, this may take a while)
Selecting by number: 21 components
Fitting ICA took 16.2s.
Fitting ICA to data using 61 channels (please be patient, this may take a while)
Selecting by number: 21 components
Fitting ICA took 12.8s.
Fitting ICA to data using 61 channels (please be patient, this may take a while)
Selecting by number: 21 components
Fitting ICA took 13.1s.
Fitting ICA to data using 61 channels (please be patient, this may take a while)
Selecting by number: 21 components
Fitting ICA took 13.8s.
Fitting ICA to data using 61 channels (please be patient, this may take a while)
Selecting by number: 21 components
Fitting ICA took 20.0s.
Fitting ICA to data using 61 channels (please be patient, this may take a while)
Selecting by number: 21 components
Fitting ICA took 20.6s.
Fitting ICA to data 

Using pre determined functions, 
detect_artifacts,
eeg_bads,
ecg_bads and
eog_bads
to mark and remove artifacts

In [18]:
#To find and mark bad components
def mark_artifacts(ica, raw):
    ica.exclude = []
    ica.detect_artifacts(raw)
    eeg_bads = list(ica.exclude)
    ecg_bads = ica.find_bads_ecg(raw)[0]
    eog_bads = ica.find_bads_eog(raw)[0]
    ica.exclude = list(set(eeg_bads+ecg_bads+eog_bads))

In [19]:
#Marking bad components
for f in files:
    mark_artifacts(ica[f], raw[f])

    Searching for artifacts...
    found 1 artifact by skewness
    found 1 artifact by kurtosis
    found 1 artifact by variance
Artifact indices found:
    5, 5, 15
    Removing duplicate indices...
Ready.
Using threshold: 0.23 for CTPS ECG detection
Using channel EKG to identify heart beats.
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 5000 samples (10.000 sec)

Number of ECG events detected : 550 (average pulse 62 / min.)
Not setting metadata
Not setting metadata
550 matching events found
No baseline correction applied
Loading data for 550 events and 501 original time points

In [20]:
#ica['10_1'].plot_sources(raw['10_1'])

In [21]:
#Applying ica
for f in files:
    ica[f].apply(raw[f], exclude = ica[f].exclude)

Applying ICA to Raw instance
    Transforming to ICA space (21 components)
    Zeroing out 4 ICA components
    Projecting back using 61 PCA components
Applying ICA to Raw instance
    Transforming to ICA space (21 components)
    Zeroing out 4 ICA components
    Projecting back using 61 PCA components
Applying ICA to Raw instance
    Transforming to ICA space (21 components)
    Zeroing out 3 ICA components
    Projecting back using 61 PCA components
Applying ICA to Raw instance
    Transforming to ICA space (21 components)
    Zeroing out 5 ICA components
    Projecting back using 61 PCA components
Applying ICA to Raw instance
    Transforming to ICA space (21 components)
    Zeroing out 3 ICA components
    Projecting back using 61 PCA components
Applying ICA to Raw instance
    Transforming to ICA space (21 components)
    Zeroing out 3 ICA components
    Projecting back using 61 PCA components
Applying ICA to Raw instance
    Transforming to ICA space (21 components)
    Zeroing o

## Generating Events for all subjects

Extracting event data from raw data

In [24]:
events = {}
for f in files:
    events[f] = mne.events_from_annotations(raw[f])

Used Annotations descriptions: ['1', '10', '11', '2', '3', '4', '5', '9']
Used Annotations descriptions: ['1', '10', '2', '3', '4', '5', '9']
Used Annotations descriptions: ['1', '10', '2', '4', '5', '6', '7', '9']
Used Annotations descriptions: ['1', '10', '11', '2', '3', '4', '5', '6', '7', '9']
Used Annotations descriptions: ['1', '10', '2', '3', '4', '5', '9']
Used Annotations descriptions: ['1', '10', '11', '2', '4', '5', '6', '7', '9']
Used Annotations descriptions: ['1', '10', '2', '4', '5', '6', '7', '9']
Used Annotations descriptions: ['1', '10', '2', '3', '4', '5', '6', '7', '9']
Used Annotations descriptions: ['1', '10', '2', '3', '4', '5', '9']
Used Annotations descriptions: ['1', '10', '2', '3', '4', '5', '6', '7', '9']
Used Annotations descriptions: ['1', '10', '11', '2', '3', '4', '5', '6', '7', '9']
Used Annotations descriptions: ['1', '10', '2', '4', '5', '6', '7', '9']


Creating a dictionary relating all events to their corresponding annotations

In [25]:
#Creating event id Dictionary
event_id = {
    "taskstart" : '9',
    "cue" : "1",
    "go" : "2",
    "button press" : "5",
    "no-go" : "4",
    "task end": "10",
    "error 1" : "3",
    "error 2" : "6",
    "error 3" : "7",
    "error 4" : "8",
    "error 5" : "11"
}
event_id

{'taskstart': '9',
 'cue': '1',
 'go': '2',
 'button press': '5',
 'no-go': '4',
 'task end': '10',
 'error 1': '3',
 'error 2': '6',
 'error 3': '7',
 'error 4': '8',
 'error 5': '11'}

## Creating Epochs

Creating epochs with time interval (-0.2, 0.5) and a baseline correction

In [44]:
#Creating epochs
epochs = {}
for f in files:
    epochs[f] = mne.Epochs(raw[f],
                   events = events[f][0],
                   event_id = events[f][1],
                           baseline = (-0.2, 0),
                           tmin = -0.2,
                           tmax = 0.5,
                          preload = True)

Not setting metadata
Not setting metadata
155 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 155 events and 351 original time points ...
0 bad epochs dropped
Not setting metadata
Not setting metadata
153 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 153 events and 351 original time points ...
0 bad epochs dropped
Not setting metadata
Not setting metadata
154 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 154 events and 351 original time points ...
0 bad epochs dropped
Not setting metadata
Not setting metadata
158 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 158 events and 351 original time points ...
0 bad epochs dropped
Not setting metadata
Not setting metadata
153 matching events found
Applying baseline correction (mode: mean)
0 proj

##### Fix Event Id
The event ids we got mne.events_from_annotation() was wrong.

For some reason It was assigning a different event id to the events.

For example, these were the ids assigned to events of 10_2

{'1': 1, '10': 2, '2': 3, '3': 4, '4': 5, '5': 6, '9': 7}

So it considered "10" as an event but assigned an id of 2 to it. We needed the event "10" to be assigned with id 10

The function fix_event _ids was a simple fix for it, which just takes the values of event array, and replace it with correct value using the dictionary.

In [45]:
#to fix event ids
def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]
def fix_event_ids(epochs, events):
    for i in range(epochs.events.shape[0]):
        epochs.events[i][2] = int(get_keys_from_value(events[1], epochs.events[i][2])[0])

In [46]:
#fix event ids
for f in files:
    fix_event_ids(epochs[f], events[f])

# Creating evoked data

Now, first we have created evoked data for different tasks(cue/go/no-go) for different subjects for both run 1 and 2. 
"1", "2" and "4" are the annotations of cue, go and no-go respectively. 

In [ ]:
evoked_101_cue = epochs['10_1']['1'].average()
evoked_101_go = epochs['10_1']['2'].average()
evoked_101_nogo = epochs['10_1']['4'].average()

evoked_102_cue = epochs['10_2']['1'].average()
evoked_102_go = epochs['10_2']['2'].average()
evoked_102_nogo = epochs['10_2']['4'].average()



evoked_121_cue = epochs['12_1']['1'].average()
evoked_121_go = epochs['12_1']['2'].average()
evoked_121_nogo = epochs['12_1']['4'].average()

evoked_122_cue = epochs['12_2']['1'].average()
evoked_122_go = epochs['12_2']['2'].average()
evoked_122_nogo = epochs['12_2']['4'].average()



evoked_131_cue = epochs['13_1']['1'].average()
evoked_131_go = epochs['13_1']['2'].average()
evoked_131_nogo = epochs['13_1']['4'].average()

evoked_132_cue = epochs['13_2']['1'].average()
evoked_132_go = epochs['13_2']['2'].average()
evoked_132_nogo = epochs['13_2']['4'].average()



evoked_281_cue = epochs['28_1']['1'].average()
evoked_281_go = epochs['28_1']['2'].average()
evoked_281_nogo = epochs['28_1']['4'].average()

evoked_282_cue = epochs['28_2']['1'].average()
evoked_282_go = epochs['28_2']['2'].average()
evoked_282_nogo = epochs['28_2']['4'].average()

evoked_311_cue = epochs['31_1']['1'].average()
evoked_311_go = epochs['31_1']['2'].average()
evoked_311_nogo = epochs['31_1']['4'].average()

evoked_312_cue = epochs['31_2']['1'].average()
evoked_312_go = epochs['31_2']['2'].average()
evoked_312_nogo = epochs['31_2']['4'].average()


evoked_321_cue = epochs['32_1']['1'].average()
evoked_321_go = epochs['32_1']['2'].average()
evoked_321_nogo = epochs['32_1']['4'].average()

evoked_322_cue = epochs['32_2']['1'].average()
evoked_322_go = epochs['32_2']['2'].average()
evoked_322_nogo = epochs['32_2']['4'].average()

In [62]:
epochs['10_1']

Number of events,155
Events,1: 6010: 4011: 22: 103: 404: 05: 09: 0
Time range,-0.200 – 0.500 sec
Baseline,-0.200 – 0.000 sec


Here we have compared the evoked plots for cue, go and no-go tasks for a single subject(AB10) for both run 1 and 2

In [ ]:
evoked_101_cue.plot(spatial_colors=True)
evoked_102_cue.plot(spatial_colors=True)

evoked_101_go.plot(spatial_colors=True)
evoked_102_go.plot(spatial_colors=True)

evoked_101_nogo.plot(spatial_colors=True)
evoked_102_nogo.plot(spatial_colors=True)

Now we combine the evoked data for all the current subjects based on the cue, go and no-go task. And now we compare this combined evoked data for these different tasks so as to get a relative idea about when the peaks occour from the baseline period and if there is a significant difference between these three different tasks

In [ ]:
evoked_CUE = mne.combine_evoked([
    evoked_101_cue,
    evoked_102_cue, 
    evoked_121_cue,
    evoked_122_cue,
    evoked_131_cue,
    evoked_132_cue,
    evoked_281_cue,
    evoked_282_cue,
    evoked_311_cue,
    evoked_312_cue,
    evoked_321_cue,
    evoked_322_cue,
    ], weights = 'equal')

evoked_GO = mne.combine_evoked([
    evoked_101_go,
    evoked_102_go, 
    evoked_121_go,
    evoked_122_go,
    evoked_131_go,
    evoked_132_go,
    evoked_281_go,
    evoked_282_go,
    evoked_311_go,
    evoked_312_go,
    evoked_321_go,
    evoked_322_go,
    ], weights = 'equal')

evoked_NOGO = mne.combine_evoked([
    evoked_101_nogo,
    evoked_102_nogo, 
    evoked_121_nogo,
    evoked_122_nogo,
    evoked_131_nogo,
    evoked_132_nogo,
    evoked_281_nogo,
    evoked_282_nogo,
    evoked_311_nogo,
    evoked_312_nogo,
    evoked_321_nogo,
    evoked_322_nogo,
    ], weights = 'equal')


evoked_CUE.plot(gfp = True)
evoked_GO.plot(gfp = True)
evoked_NOGO.plot(gfp = True)
# mne.viz.plot_compare_evokeds(
#     [
#         evoked_101_cue,
#         evoked_101_go,
#         evoked_diff
#     ]
# )

In [ ]:
mne.viz.plot_compare_evokeds(
    [
        evoked_CUE,
        evoked_GO,
        evoked_NOGO
    ],
    legend=True
)

[<Figure size 800x600 with 1 Axes>]

Now we plot the difference between cue/go, cue/no-go and go/no-go evoked plots. That is why the weights have been set to 1, -1

In [ ]:
diff_evoked_CUEGO = mne.combine_evoked([evoked_CUE, evoked_GO], weights=[1, -1])
diff_evoked_CUENOGO = mne.combine_evoked([evoked_CUE, evoked_NOGO], weights=[1, -1])
diff_evoked_GONOGO = mne.combine_evoked([evoked_GO, evoked_NOGO], weights=[1, -1])

In [ ]:
mne.viz.plot_compare_evokeds(
    [
        evoked_CUE,
        evoked_GO,
        diff_evoked_CUEGO
    ],
    legend=True
)

[<Figure size 800x600 with 1 Axes>]

In [ ]:
mne.viz.plot_compare_evokeds(
    [
        evoked_GO,
        evoked_NOGO,
        diff_evoked_CUENOGO
    ],
    legend=True
)

[<Figure size 800x600 with 1 Axes>]

In [ ]:
mne.viz.plot_compare_evokeds(
    [
        evoked_GO,
        evoked_NOGO,
        diff_evoked_GONOGO
    ],
    legend=True
)

[<Figure size 800x600 with 1 Axes>]

To plot averages of each event

## Preparing data for models

The below variable datadic is just a dictionary storing the data matrix of the various files (like '10_1' and '10_2') as key-value pairs. We get the data by using the .get_data() function and picking only the EEG components and then finally storing them in the datadic variable.

In [50]:
#Getting epoch data
datadic = {}
for f in files:
    datadic[f] = epochs[f].get_data(picks = 'eeg')

Below we have just reshaped the input matrix from (num_samples X num_channels X time_points_per_epoch) to (num_samples X time_points_per_epoch X num_channels). So the initial dimensions from (1875 X 61 X 351) have now been changed to (1875 X 351 x 61).

In [51]:
#Concatenating data
data = np.concatenate(list(datadic.values()), 
                      axis = 0)
print(data.shape)

#Changing the shape of data from (events, channel, time points) to (events, time points, channel)
datars = np.zeros((data.shape[0], data.shape[2], data.shape[1]))
for i in range(datars.shape[0]):
    datars[i] = np.transpose(data[i])
    
dims_lstm_1 = datars.shape[1]
dims_lstm_2 = datars.shape[2]
print(dims_lstm_1, "X", dims_lstm_2)
datars.shape

(1875, 61, 351)
351 X 61


(1875, 351, 61)

Just like datadic above, the ydic variable stores the various output class values for all the subjects per run as key-value pairs. Then we create y by just joining the values for the various keys to create the whole 1D y matrix. 

In [55]:
#Getting y
ydic = {}
for f in files:
    ydic[f] = epochs[f].events[:, 2]
y = np.concatenate(list(ydic.values()), axis = 0)
y = y.reshape(-1, 1)

##### Saving data in a .npz file so as to be used by other notebooks

In [56]:
np.savez('data1.npz', X = datars, y = y)